# Libaries

In [1]:
!pip install transformers datasets transformers[torch]
!git clone https://github.com/TrungGa1306/DuAnCNTT.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 23.3 MB/s eta 0:00:00
Cloning into 'DuAnCNTT'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (4/4), 9.04 MiB | 16.75 MiB/s, done.


In [17]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig
from transformers import Trainer, TrainingArguments , EarlyStoppingCallback
from transformers import pipeline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Reading, Processing data

In [3]:
file_name = '/content/DuAnCNTT/vietnamese_history_v2.jsonl'

# Đọc file data
VnHistory_dataset = load_dataset("json", data_files = file_name, split = "train")
VnHistory_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['page', 'book', 'context'],
    num_rows: 13122
})

In [4]:
# Lấy sample
VnHistory_dataset[15]

{'page': '35',
 'book': 'lich-su-viet-nam-tron-bo-15-tap-pdf-tap-4',
 'context': "Chương l. Sự phân liệt Đàng Trong - Đàng Ngoài... Đồi với họ Trịnh, thời gian đầu vì còn phải tập trung vào diệt trừ kẻ thù chung là triều Mạc, Nguyễn Hoàng ngoài việc thực hiện đầy đủ nghĩa vụ thuế khóa đã giữ được quan hệ thân thuộc và tỏ ra hết sức trung thành với vua Lê, chúa Trịnh, khiến họ Trịnh không mấy nghi ngờ. Năm 1570', Trịnh Kiểm mất, quyền hành trao cho Trịnh Tùng, nội bộ Nam triều xảy ra lục đục, quân Mạc mở nhiều cuộc tấn công vào vùng Thanh - Nghệ, vua Lê phải gọi Nguyễn Bá Quỳnh vừa thay Bùi Tá Hán làm trấn thủ Quảng Nam ra làm trần thủ Nghệ An, rồi giao cho Nguyễn Hoàng làm trấn thủ luôn cả hai xứ Thuận Hóa và Quảng Nam với chức Tổng trấn tướng quân . Làm Tỏổng trấn tướng quân, kiêm lĩnh hai xứ Thuận Hóa và Quảng Nam, Nguyễn Hoàng đã cho chuyển dinh thự từ xã Á¡ Tử đến xã 7rà Bát thuộc huyện Vũ Xương (Triệu Phong, Quảng Trị). Còn về tổ chức chính quyền thì Nguyễn Hoàng vẫn giữ nguyên nh

In [5]:
# In thường phần context , giữ cột context
def lowercase_condition(datasets):
  context = ""
  for token in datasets["context"]:
    context += token.lower()
  return {"context": context}

VnHistory_dataset = VnHistory_dataset.map(lowercase_condition , num_proc=8 , remove_columns= VnHistory_dataset.column_names )
VnHistory_dataset[15]

Map (num_proc=8):   0%|          | 0/13122 [00:00<?, ? examples/s]

{'context': "chương l. sự phân liệt đàng trong - đàng ngoài... đồi với họ trịnh, thời gian đầu vì còn phải tập trung vào diệt trừ kẻ thù chung là triều mạc, nguyễn hoàng ngoài việc thực hiện đầy đủ nghĩa vụ thuế khóa đã giữ được quan hệ thân thuộc và tỏ ra hết sức trung thành với vua lê, chúa trịnh, khiến họ trịnh không mấy nghi ngờ. năm 1570', trịnh kiểm mất, quyền hành trao cho trịnh tùng, nội bộ nam triều xảy ra lục đục, quân mạc mở nhiều cuộc tấn công vào vùng thanh - nghệ, vua lê phải gọi nguyễn bá quỳnh vừa thay bùi tá hán làm trấn thủ quảng nam ra làm trần thủ nghệ an, rồi giao cho nguyễn hoàng làm trấn thủ luôn cả hai xứ thuận hóa và quảng nam với chức tổng trấn tướng quân . làm tỏổng trấn tướng quân, kiêm lĩnh hai xứ thuận hóa và quảng nam, nguyễn hoàng đã cho chuyển dinh thự từ xã á¡ tử đến xã 7rà bát thuộc huyện vũ xương (triệu phong, quảng trị). còn về tổ chức chính quyền thì nguyễn hoàng vẫn giữ nguyên những đơn vị hành chính cũ của họ trịnh. thuận hóa vẫn bao gồm: 2 phủ, 

In [6]:
VnHistory_dataset

Dataset({
    features: ['context'],
    num_rows: 13122
})

# Training a new tokenizer from GPT2 tokenizer

In [7]:
# Gọi pretrained tokenizer của gpt2
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(old_tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [8]:
load_tokenize = True
tokenizer_checkpoint = "/content/drive/MyDrive/DuAnCNTT/Source_code/VnHistory_tokenizer"

if load_tokenize:
  tokenizer =  AutoTokenizer.from_pretrained(tokenizer_checkpoint)
else:
  # Training tokenizer mới dựa trên bộ dataset
  tokenizer = old_tokenizer.train_new_from_iterator(VnHistory_dataset["context"], 70000)
  tokenizer.save_pretrained(tokenizer_checkpoint)

In [9]:
# Kiểm tra tokenizer
sample = "Lịch sử Việt Nam tập 9. Hồ Chí Minh là người sáng lập Đảng Cộng sản Việt Nam, từng là Chủ tịch nước Việt Nam Dân chủ Cộng hoà từ 1945–1969, Thủ tướng Việt Nam Dân chủ Cộng hòa trong những năm 1945–1955, Tổng Bí thư Ban Chấp hành Trung ương Đảng Lao động Việt Nam từ 1956–1960, Chủ tịch Ban Chấp hành Trung ương Đảng Lao động Việt Nam từ năm 1951 cho đến khi qua đời."
sample = tokenizer(" ".join(words.lower() for words in sample.split()))
print(sample)
print(tokenizer.decode(sample["input_ids"]))
print(tokenizer.bos_token , tokenizer.eos_token , tokenizer.pad_token)

{'input_ids': [898, 496, 429, 374, 509, 818, 14, 675, 780, 654, 307, 364, 1301, 630, 756, 892, 596, 429, 374, 12, 1198, 307, 536, 1404, 425, 429, 374, 403, 536, 892, 605, 439, 2388, 444, 242, 4672, 12, 641, 760, 429, 374, 403, 536, 892, 915, 373, 413, 376, 2388, 444, 242, 3477, 12, 850, 1602, 724, 791, 1739, 637, 508, 1261, 756, 1143, 582, 429, 374, 439, 4117, 444, 242, 3745, 12, 536, 1404, 791, 1739, 637, 508, 1261, 756, 1143, 582, 429, 374, 439, 376, 2894, 359, 410, 528, 819, 807, 14], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
lịch sử việt nam tập 9. hồ chí minh là người sáng lập đảng cộng sản việt nam, từng là chủ tịch nước việt nam dân chủ cộng hoà từ 1945–1969, thủ tướng việt nam dân chủ cộng hòa trong những năm 1945–1955, tổng 

# Building tokenizer dataset, datacollator and Initializing GPT2LMHeadModel

<h2>Tạo tokenizer dataset</h2>

In [10]:
context_length = 1024
stride = 256

def tokenize(element):
    outputs = tokenizer(
        element["context"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
        stride = stride,
    )
    outputs['input_ids'] = outputs['input_ids']

    return outputs
# Tạo tokenizer dataset
VnHistory_tokenizer_dataset = VnHistory_dataset.map(tokenize, batched=True
                                                    ,remove_columns=VnHistory_dataset.column_names)\
                                                .train_test_split(test_size=0.1, shuffle=True, seed=123)

Map:   0%|          | 0/13122 [00:00<?, ? examples/s]

In [11]:
VnHistory_tokenizer_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 11811
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 1313
    })
})

<h2>Tạo datacollator</h2>

In [12]:

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2TokenizerFast(name_or_path='/content/drive/MyDrive/DuAnCNTT/Source_code/VnHistory_tokenizer', vocab_size=55337, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

<h2>Tạo kiến trúc mô hình</h2>

In [13]:


config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [14]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 128.3M parameters


# Pretrain GPT2LMHeadModel

In [15]:
load_model = True
pretrain = False
output_dir = "/content/drive/MyDrive/DuAnCNTT/Source_code/model_checkpoints/gpt2_for_VnHistory"
batch_size = 3

if load_model:
  print("Loading local pretrained model checkpoint.")
  model = GPT2LMHeadModel.from_pretrained(output_dir+"/checkpoint-4000")

args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="steps",
    eval_steps=1_000,
    logging_steps=1_000,
    gradient_accumulation_steps=8,
    num_train_epochs=30,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=2_000,
    fp16=True,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end = True,
)

early_stopping = EarlyStoppingCallback(early_stopping_patience= 25,
                                    early_stopping_threshold= 0.001,
                                    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=VnHistory_tokenizer_dataset["train"],
    eval_dataset=VnHistory_tokenizer_dataset["test"],
    callbacks= [early_stopping,],
)

Loading local pretrained model checkpoint.


In [16]:
if pretrain:
  trainer.train()

<h2>PipeLine local pretrained model

In [58]:
# Thực hiện pipeline mô hình pretrain
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation",
    model="/content/drive/MyDrive/DuAnCNTT/Source_code/model_checkpoints/gpt2_for_VnHistory/checkpoint-4000",
    device=device,
)

print(pipe("Nước Lâm Áp quấy nhiễu Giao Châu"))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Nước Lâm Áp quấy nhiễu Giao Châu như cũ xảy ra vào năm man, chín châu quận có lý do là lang cháy nóng nảy, dẫu có kẻ nào khác được nhờ quyền thế thậm chí ngày nay ư'}]


In [52]:
pipe = pipeline(
    "question-answering",
    model="/content/drive/MyDrive/DuAnCNTT/Source_code/model_checkpoints/gpt2_for_VnHistory/checkpoint-4000",
    device=device,
)

question = "Chủ tịch Hồ Chí Minh là ai?"
context = "Chủ tịch Hồ Chí Minh, tên thật là Nguyễn Sinh Cung, sau này được biết đến với tên gọi tôn kính là Bác Hồ, là một lãnh tụ vĩ đại của dân tộc Việt Nam và một danh nhân văn hóa thế giới. Ông được xem như một người cha già của dân tộc, có công lớn trong việc tìm đường cứu nước và đưa nhân dân thoát khỏi cuộc sống đầy khổ đau và bị áp bức."

print(pipe(question = question, context = context,))

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at /content/drive/MyDrive/DuAnCNTT/Source_code/model_checkpoints/gpt2_for_VnHistory/checkpoint-4000 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.00426828907802701, 'start': 144, 'end': 160, 'answer': ' Nam và một danh'}
